In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from scipy.stats import norm
import sklearn.metrics as metrics
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
data   =   pd.read_csv( 'penguins_size.csv' )   
ss   =   StandardScaler ()

In [3]:
data   =   data.dropna()   
data   =   data.drop([ 'sex' ,    'island' ,    'flipper_length_mm' ,    'body_mass_g' ],   axis= 1)

In [4]:
#   Prepare   input   
X   =   data.drop([ 'species' ],   axis= 1 )  
columns   =   X.columns   
X   =   X.values   
X   =   ss.fit_transform(X)   

In [5]:
#   Prepare   target   
y   =   data[ 'species' ]   
species   =   { 'Adelie' :    0 ,    'Chinstrap' :    1 ,    'Gentoo' :    2 }   
y   =   [species[item]    for    item    in    y]   
y   =   np.array(y)    

In [6]:
#   Split   the   data   
X_train,   X_test,   y_train,   y_test   =   train_test_split(X,   y,   test_size= 0.2 ,   random_state= 33 )  

In [11]:
class     MyNaiveBias ():         
    '''Implements   algorithm   for   Naive   Bias'''         
    def     __init__ (self,   input_columns):            
        self.input_columns   =   input_columns             
    
    def     fit (self,   X_train,   y_train):            
        X_train   =   pd.DataFrame(X_train,   columns   =   self.input_columns)
        self.classes   =   np.unique(y_train)            
        self.means   =   X_train.groupby(y_train).apply(np.mean)            
        self.stds   =   X_train.groupby(y_train).apply(np.std)            
        self.probabilities   =   X_train.groupby(y_train).apply( lambda    x:   len(x))   /   X_train.shape[ 0 ]           
    
    def predict (self,   X_test):            
        X_test   =   pd.DataFrame(X_test,   columns   =   self.input_columns)            
        predictions   =   []                
        
        for    i    in    range(X_test.shape[ 0 ]):               
            p   =   {}                     
            
            for    c    in    self.classes:                    
                p[c]   =   self.probabilities[c]                        
                
                for    index,   row    in    enumerate(X_test.iloc[i]):                        
                    p[c]   *=   norm.pdf(row,   self.means.iloc[c,   index],   self.stds.iloc[c,   index])                    
                    
            predictions.append(pd.Series(p).values.argmax())         
            
        return predictions

In [13]:
model   =   MyNaiveBias(columns)     

model.fit(X_train,   y_train)   
my_naive_predictions=   model.predict(X_test)     

print(metrics.classification_report(y_test,   my_naive_predictions))     

pd.DataFrame({        
    'Actual   Value' :   y_test,         
    'Naive   Bias   Predictions' :   my_naive_predictions,   
})

              precision    recall  f1-score   support

           0       0.96      1.00      0.98        23
           1       0.91      0.77      0.83        13
           2       0.94      0.97      0.95        31

    accuracy                           0.94        67
   macro avg       0.93      0.91      0.92        67
weighted avg       0.94      0.94      0.94        67



C:\Users\adrim\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)


,Actual Value,Naive Bias Predictions
0,1,1
1,2,2
2,0,0
3,2,1
4,0,0
...,...,...
62,2,2
63,2,2
64,2,2
65,0,0


In [18]:
model   =   GaussianNB()   
model.fit(X_train,   y_train)   
sk_nb_preditions   =   model.predict(X_test) 
predictions = model.predict(X_test)

print(accuracy_score(sk_nb_preditions,   y_test))   

pd.DataFrame({  
    'Actual   Value' :   y_test,  
    'Naive   Bias   Predictions' :   predictions,  
    'SciKit   Learn   NB' :   sk_nb_preditions,   
})   

0.9402985074626866


,Actual Value,Naive Bias Predictions,SciKit Learn NB
0,1,1,1
1,2,2,2
2,0,0,0
3,2,1,1
4,0,0,0
...,...,...,...
62,2,2,2
63,2,2,2
64,2,2,2
65,0,0,0
